In [2]:
import joblib
from sklearn.preprocessing import StandardScaler , MinMaxScaler
from sklearn.model_selection import LeaveOneOut
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
# from envs.filtr_json_from_race import load_from_db
import sqlite3
import json
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score

In [11]:
class LSTMStatePredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMStatePredictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.scaler_X = None
        self.scaler_Y = None

    def forward(self, x):
        h_0 = torch.zeros(1, x.size(0), self.lstm.hidden_size).to(x.device) #hidden state
        c_0 = torch.zeros(1, x.size(0), self.lstm.hidden_size).to(x.device) #cell state

        out, _ = self.lstm(x, (h_0, c_0))
        out = self.fc(out[:, -1, :])  # Use the last time step's output
        return out

def create_scalers(X,Y):

    cont_indices_x = slice(0, 18)   # continuous columns for X (0–17)
    cont_indices_y = slice(0, 12)   # continuous columns for Y (0–11)

    # Scale continuous features
    flat_x = np.vstack([x[:, cont_indices_x] for x in X])
    flat_y = np.vstack([y[:, cont_indices_y] for y in Y])

    scaler_X = MinMaxScaler().fit(flat_x)
    scaler_Y = MinMaxScaler().fit(flat_y)
    return scaler_X, scaler_Y


def scale_input(X, Y, scaler_X, scaler_Y):
    cont_indices_x = slice(0, 18)   # continuous columns for X
    cont_indices_y = slice(0, 12)   # continuous columns for Y

    X_scaled_grouped = []
    Y_scaled_grouped = []

    for x_seq, y_seq in zip(X, Y):
        x_scaled = np.array(x_seq, dtype=float)
        x_scaled[:, cont_indices_x] = scaler_X.transform(x_seq[:, cont_indices_x])
        X_scaled_grouped.append(x_scaled)

        y_scaled = np.array(y_seq, dtype=float)
        y_scaled[:, cont_indices_y] = scaler_Y.transform(y_seq[:, cont_indices_y])
        Y_scaled_grouped.append(y_scaled)

    # Conversion to torch tensors
    # X_t = [torch.tensor(x, dtype=torch.float32) for x in X_scaled_grouped]
    # Y_cont_t = [torch.tensor(y[:, cont_indices_y], dtype=torch.float32) for y in Y_scaled_grouped]

    return X_scaled_grouped, Y_scaled_grouped

In [17]:
def load_data_from_db():
    
    """
    Load data so that each race is a separate sequence:
    X = [ [state1_race1, state2_race1, ...], [state1_race2, ...] ]
    Y = [ [next1_race1, next2_race1, ...], ... ]
    """
    conn = sqlite3.connect(
        "E:/pracadyp/Race-optimization-reinforcement-learning/data/db_states_for_regress/race_data_states.db"
    )
    cursor = conn.cursor()
    cursor.execute("SELECT race_id, states_json FROM races ORDER BY race_id")
    rows = cursor.fetchall()
    conn.close()


    data = []

    for race_id, states_json in rows:
        states = json.loads(states_json)
        data.append(states)
    
    return data

def create_windows(sequence_x, sequence_y, window_size, n_steps_ahead=5):
    X, Y = [], []
    for t in range(1, len(sequence_x)):
        start = max(0, t - window_size)
        window = sequence_x[start:t]

        # padding na początku, jeśli okno krótsze niż window_size
        pad_len = window_size - len(window)
        if pad_len > 0:
            window = np.vstack([np.zeros((pad_len, sequence_x.shape[1])), window])
        X.append(window)

        # Y: wypełniamy zerami, jeśli końcówka wyścigu ma mniej niż n_steps_ahead
        y_window = sequence_y[t:t+n_steps_ahead]
        if y_window.shape[0] < n_steps_ahead:
            pad = np.zeros((n_steps_ahead - y_window.shape[0], sequence_y.shape[1]))
            y_window = np.vstack([y_window, pad])
        Y.append(y_window)

    return np.array(X), np.array(Y)

def create_x_y(data):
    X_grouped, Y_grouped = [], []

    for race in data:
        X_seq, Y_seq = [], []
        for i in range(len(race) - 1):
            X_seq.append(race[i][:-2])
            Y_seq.append(race[i + 1][:-24])
        
        # dodajemy każdy wyścig osobno
        X_grouped.append(np.array(X_seq, dtype=float))
        Y_grouped.append(np.array(Y_seq, dtype=float))

    return X_grouped, Y_grouped

In [ ]:
data = load_data_from_db()
# input_size = X[0].shape[1]
# output_size = Y[0].shape[1]

X, Y = create_x_y(data)
input_size = X[0].shape[1]
output_size = Y[0].shape[1]

scaler_X, scaler_Y = create_scalers(X,Y)

X_scaled, Y_scaled = scale_input(X,Y,scaler_X,scaler_Y)

# X_scaled = [scaler_X.transform(np.array(race)) for race in X]
# Y_scaled = [scaler_Y.transform(np.array(race)) for race in Y]


loo = LeaveOneOut()
all_fold_train = []
all_fold_test = []
all_fold_r2_test = []
for fold, (train_idx, test_idx) in enumerate(loo.split(X)):

    X_train = [X[i] for i in train_idx]
    X_test  = [X[i] for i in test_idx]
    Y_train = [Y[i] for i in train_idx]
    Y_test  = [Y[i] for i in test_idx]


    all_X, all_Y = [], []
    for race_x, race_y in zip(X_train, Y_train):  
        X_r, Y_r = create_windows(race_x, race_y, window_size=30)
        all_X.append(X_r)
        all_Y.append(Y_r)

    X_train = np.vstack(all_X)  # shape: [N_samples, window_size, n_features]
    Y_train = np.vstack(all_Y) 
    all_X, all_Y = [], []
    for race_x, race_y in zip(X_test, Y_test):  
        X_r, Y_r = create_windows(race_x, race_y, window_size=30)
        all_X.append(X_r)
        all_Y.append(Y_r)
    X_test = np.vstack(all_X)  # shape: [N_samples, window_size, n_features]
    Y_test = np.vstack(all_Y)


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    model = LSTMStatePredictor(input_size=input_size, hidden_size=128, output_size=output_size, num_layers=1).to(device)



    
    
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32).to(device)

    train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)

    optimizer = optim.Adam(model.parameters(), lr=4e-5)
    loss_cont = nn.MSELoss()
    loss_cat  = nn.CrossEntropyLoss()

    fold_train_losses = []
    fold_test_losses = []


    



Using device: cuda


AttributeError: 'LSTMStatePredictor' object has no attribute 'scale_input'